In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [3]:
## reading csv file
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Preprocessing the data
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [5]:
## encode categorical variables
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

In [6]:
## one-hot encode Geography
onehot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=onehot_encoder_geo.fit_transform(df[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
## combine one-hot encoded column with original data
df=pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
## split the data into features and target
X=df.drop('EstimatedSalary',axis=1)
y=df['EstimatedSalary']

In [9]:
## split the data for training and testing
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=40)

## scaling the features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)


In [10]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [11]:
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


### ANN Regression

In [12]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 


In [13]:
## Building ANN Model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),  ## HL1 CONNECTED TO INPUT LAYER
    Dense(32,activation='relu'), ## HL2
    Dense(1)  ## Output layer
]
)

In [15]:
## compile the model
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

##setup Tensor Board
log_dir="regressionlogs/fit/"+ datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [17]:
## set up early stoppings
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [18]:
## Train the model
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100


250/250 [==============================] - 5s 10ms/step - loss: 100206.6172 - mae: 100206.6172 - val_loss: 99313.4688 - val_mae: 99313.4688
Epoch 2/100
250/250 [==============================] - 1s 6ms/step - loss: 99649.4844 - mae: 99649.4844 - val_loss: 98179.4688 - val_mae: 98179.4688
Epoch 3/100
250/250 [==============================] - 2s 8ms/step - loss: 97681.0859 - mae: 97681.0859 - val_loss: 95246.6562 - val_mae: 95246.6562
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 93708.6562 - mae: 93708.6562 - val_loss: 90184.2656 - val_mae: 90184.2656
Epoch 5/100
250/250 [==============================] - 1s 6ms/step - loss: 87673.8516 - mae: 87673.8516 - val_loss: 83271.7812 - val_mae: 83271.7812
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 80118.4688 - mae: 80118.4688 - val_loss: 75385.0234 - val_mae: 75385.0234
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 71991.5156 - mae: 71991.51

In [19]:
%load_ext tensorboard

In [24]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 7060), started 0:00:23 ago. (Use '!kill 7060' to kill it.)

In [25]:
## evaluate the model
test_loss,test_mae=model.evaluate(X_test,y_test)
print(f'Test MAE :{test_mae}')

63/63 [==============================] - 0s 3ms/step - loss: 50788.3594 - mae: 50788.3594
Test MAE :50788.359375


In [26]:
model.save('regression_model.h5')

d:\DL Project\ANN Classification\dlvenv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
